In [ ]:
#Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from PIL import Image
import clip

In [ ]:
data = pd.read_csv('cat_dog1.csv')
data

,Unnamed: 0,image,labels
0,0,dog.624.jpg,1
1,1,cat.170.jpg,0
2,2,dog.419.jpg,1
3,3,dog.19.jpg,1
4,4,cat.227.jpg,0
...,...,...,...
1995,1995,dog.481.jpg,1
1996,1996,dog.159.jpg,1
1997,1997,cat.576.jpg,0
1998,1998,dog.190.jpg,1


In [ ]:
#a-Spliting the dataset into training and testing sets with 20% for test dataset
#made it 0.9 for ease of testing for now
train_data, test_data = train_test_split(data, test_size=0.9, random_state=42)

In [ ]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-l2scypmu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-l2scypmu
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=456e438ebf8efc9d60537cebbbadb4635931084c8ed44289c540a2da1211a34c
  Stored in directory: /tmp/pip-ephem-wheel-cache-b9vv04jx/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 106MiB/s]


In [ ]:
#Function to get CLIP embeddings for an image
def get_clip_embedding(image_path):
    image = transform(Image.open(image_path)).unsqueeze(0).to(device)
    embedding = model.encode_image(image)
    return embedding

#Example of using the function on a single image
image_path = "cat-dog/cat.0.jpg"
embedding = get_clip_embedding(image_path)

In [ ]:
def get_feature_matrix(dataset):
    embeddings = [get_clip_embedding(image_path) for image_path in dataset['image_path']]
    feature_matrix = torch.stack(embeddings).detach().cpu().numpy()
    return feature_matrix

# Apply the function to both training and testing datasets
train_feature_matrix = get_feature_matrix(train_data)
test_feature_matrix = get_feature_matrix(test_data)